# Deadlock Meta Analytics: Hypothesis Testing

This notebook analyzes match data to test key hypotheses about game balance and mechanics.

## Hypotheses
1. **Late Game Heroes**: Does hero winrate change with match duration?
2. **Snowball Effect**: Is there a strong correlation between Net Worth lead and victory?
3. **Meta Tier List**: Which heroes are the most effective in the current patch?

In [ ]:
import sys
import os
import pandas as pd
import plotly.express as px

# Add project root to path to import src modules
sys.path.append(os.path.join(os.getcwd(), '..'))

from src.analysis.analytics import DeadlockAnalyzer

analyzer = DeadlockAnalyzer(db_path=os.path.abspath(os.path.join('..', 'deadlock.db')))
print("Analyzer initialized.")

## 1. Late Game Heroes
Hypothesis: Some heroes (like Haze) become stronger as the game goes on, while others fall off.

In [ ]:
df_duration = analyzer.get_winrate_by_duration()
print(f"Data points: {len(df_duration)}")
df_duration.head()

In [ ]:
# Filter for top played heroes to avoid noise
top_heroes = df_duration.groupby('hero_id')['matches_played'].sum().nlargest(5).index
df_filtered = df_duration[df_duration['hero_id'].isin(top_heroes)]

fig = px.line(
    df_filtered, 
    x='duration_bucket', 
    y='winrate', 
    color='hero_id', 
    title='Winrate by Match Duration (Top 5 Heroes)',
    category_orders={'duration_bucket': ['0-15m', '15-25m', '25-40m', '40m+']}
)
fig.show()

## 2. Snowball Effect
Hypothesis: Teams with a significant Net Worth lead usually win.

In [ ]:
df_snowball = analyzer.get_net_worth_correlation()
df_snowball.head()

In [ ]:
fig = px.box(
    df_snowball, 
    x='is_winner', 
    y='avg_team_net_worth', 
    color='is_winner',
    title='Net Worth Distribution: Winners vs Losers'
)
fig.show()

## 3. Meta Tier List
Hypothesis: The best heroes have both high pick rates and high win rates.

In [ ]:
df_meta = analyzer.get_meta_rating()
df_meta.head(10)

In [ ]:
fig = px.scatter(
    df_meta, 
    x='pick_rate', 
    y='win_rate', 
    size='meta_score', 
    color='meta_score', 
    hover_data=['hero_id'],
    title='Meta Analysis: Pick Rate vs Win Rate'
)
fig.show()